In [38]:
import pandas as pd

# Importing CSV Files
The CSV files are from Wireshark on attack and normal sessions that was created.

In [39]:
dfAttack = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpattack.csv")

In [40]:
dfAttack2 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpattack2.csv")

In [41]:
dfAttack3 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpattack3.csv")

In [42]:
dfAttack4 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpattack4.csv")

In [43]:
dfAttack5 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpattack5.csv")

In [44]:
dfAttack6 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpattack6.csv")

------------------------------------------------------

In [45]:
dfNormal = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpnormal.csv")

In [46]:
dfNormal2 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpnormal2.csv")

In [47]:
dfNormal3 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpnormal3.csv")

In [48]:
dfNormal4 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpnormal4.csv")

In [49]:
dfNormal5 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpnormal5.csv")

In [50]:
dfNormal6 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpnormal6.csv")

In [51]:
dfNormal7 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpnormal7.csv")

--------------------------------------------------------------------------------

# Label the dataframes accordingly based on what the traffic is
Since the CSV files are separated, we can label the attack session with 1 and the normal session for 0. 

In [52]:
dfAttack6["Label"] = 1
dfAttack5["Label"] = 1
dfAttack4["Label"] = 1
dfAttack3["Label"] = 1
dfAttack2["Label"] = 1
dfAttack["Label"] = 1

dfNormal["Label"] = 0
dfNormal2["Label"] = 0
dfNormal3["Label"] = 0
dfNormal4["Label"] = 0
dfNormal5["Label"] = 0
dfNormal6["Label"] = 0
dfNormal7["Label"] = 0

##### Quick validation count for each rows in the dataframes
To check if the dataset is imbalance with the attack session and normal session.

In [53]:
data = [dfAttack, dfAttack2, dfAttack3, dfAttack4, dfAttack5, dfAttack6, 
        dfNormal, dfNormal2, dfNormal3, dfNormal4, dfNormal5, dfNormal6, dfNormal7]

for i in data:
    num_rows = len(i)
    print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 1393
Number of rows in the DataFrame: 3442
Number of rows in the DataFrame: 1413
Number of rows in the DataFrame: 85
Number of rows in the DataFrame: 31
Number of rows in the DataFrame: 155
Number of rows in the DataFrame: 127
Number of rows in the DataFrame: 38
Number of rows in the DataFrame: 130
Number of rows in the DataFrame: 927
Number of rows in the DataFrame: 1233
Number of rows in the DataFrame: 3700
Number of rows in the DataFrame: 330


------------------------------------------------------------------------

# Join the normal and attack session dataframes
Combine all dataframes into one and remove any rows that are null

In [54]:
df = pd.concat(data, axis=0, ignore_index=True)

# Drop rows that are null
df.dropna(inplace=True)

In [55]:
df.loc[df['Full request URI'].str.contains('login.php'), 'Label'] = 0
df.loc[df['Full request URI'].str.contains('setup.php'), 'Label'] = 0

In [56]:
# Check if the ratio of attack(1) and normal(0) entry on the dataset needs to be balanced from one way to another
value_counts = df['Label'].value_counts()
print(value_counts)

0    6503
1    6501
Name: Label, dtype: int64


In [57]:
common_values = df['Full request URI'].value_counts()
common_values = common_values[common_values > 1].index.tolist()

# Count occurrences of each common value
for value in common_values:
    count = df[df['Full request URI'] == value].shape[0]
    print(f"'{value}' occurs {count}")


'http://detectportal.firefox.com/success.txt?ipv4' occurs 12
'http://192.168.95.129/dvwa/login.php' occurs 11
'http://192.168.95.129/dvwa/vulnerabilities/upload/' occurs 10
'http://172.24.30.219/dvwa/vulnerabilities/fi/?page=include.php' occurs 10
'http://172.24.30.219/dvwa/vulnerabilities/brute/' occurs 10
'http://192.168.95.129/dvwa/vulnerabilities/fi/?page=include.php' occurs 10
'http://172.24.30.219/dvwa/vulnerabilities/csrf/' occurs 9
'http://ocsp.starfieldtech.com//MEQwQjBAMD4wPDAJBgUrDgMCGgUABBSLwZ6EW5gdYc9UaSEaaLjjETNtkAQUv1%2B30c7dH4b0W1Ws3NcQwg6piOcCAzkUhA%3D%3D' occurs 9
'http://172.24.30.219/dvwa/phpinfo.php?=PHPE9568F35-D428-11d2-A769-00AA001ACF42' occurs 9
'http://172.24.30.219/dvwa/phpinfo.php?=SUHO8567F54-D428-14d2-A769-00DA302A5F18' occurs 9
'http://172.24.30.219/dvwa/phpinfo.php' occurs 9
'http://172.24.30.219/dvwa/phpinfo.php?=PHPE9568F34-D428-11d2-A769-00AA001ACF42' occurs 9
'http://192.168.95.129/dvwa/instructions.php' occurs 8
'http://192.168.95.129/dvwa/vulnerabi

'http://192.168.254.113/mutillidae/index.php?page=%2e%2e%2f%2e%2e%2f%2e%2e%2f%2e%2e%2f%2e%2e%2f%2e%2e%2fboot%2eini%3bindex%2ehtml' occurs 4
'http://192.168.254.113/mutillidae/index.php?page=%2e%2e%2f%2e%2e%2f%2e%2e%2f%2e%2e%2f%2e%2e%2f%2e%2e%2f%2e%2e%2f%2e%2e%2fboot%2eini%3bindex%2ehtml' occurs 4
'http://www.news.cn/2023homepro/images/talk-bg.png' occurs 4
'http://wza.news.cn/dist/aria.js?appid=7c11fb3e5d5e6bc07fc782d63ab71188' occurs 4
'http://www.news.cn/2023homepro/images/accessible.png' occurs 4
'http://a2.xinhuanet.com/download/adcreatives/2023/9/1/202309011149329978.png' occurs 4
'http://www.news.cn/2023homepro/images/xhwindow-icon4.png' occurs 4
'http://a2.xinhuanet.com/download/adcreatives/2023/9/11/202309110409569623.jpg' occurs 4
'http://www.news.cn/2023homepro/images/talk-scroll-bg.png' occurs 4
'http://192.168.254.113/mutillidae/index.php?page=%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5cboot%2eini' occurs 4
'http://192.168.254.113/mutillidae/index.php?page=%2e%2e%5c%2e%2e%

'http://channelpic.360doc.com/2022/20221127/20221127900629125699.jpg' occurs 2
'http://channelpic.360doc.com/2022/20221118/2022111817369769106783.jpg' occurs 2
'http://channelpic.360doc.com/2022/20221223/20221223183029301112616.jpg' occurs 2
'http://channelpic.360doc.com/2022/20221218/20221218223219274117640.jpg' occurs 2
'http://channelpic.360doc.com/2022/20221209/2022129165530146219125.jpg' occurs 2
'http://channelpic.360doc.com/2022/20221203/202212315445760079972.jpg' occurs 2
'http://channelpic.360doc.com/2022/20221127/2022112790064491601.jpg' occurs 2
'http://people.com.cn/' occurs 2
'http://channelpic.360doc.com/2022/20221223/20221223183029286151106.jpg' occurs 2
'http://channelpic.360doc.com/2022/20221218/20221218223219289131678.jpg' occurs 2
'http://channelpic.360doc.com/2022/20221209/2022129165530160134368.jpg' occurs 2
'http://channelpic.360doc.com/2022/20221209/2022129165530129163808.jpg' occurs 2
'http://channelpic.360doc.com/2022/20221203/2022123154457584200637.jpg' occurs

'http://targetedattacks.trendmicro.com/img/flags/Thai.png' occurs 2
'http://targetedattacks.trendmicro.com/wp-content/themes/starter_theme/images/fugle_footer_04.png' occurs 2
'http://targetedattacks.trendmicro.com/wp-content/themes/starter_theme/images/fugle_footer_03.png' occurs 2
'http://targetedattacks.trendmicro.com/wp-content/themes/starter_theme/images/by-trendmicro.png' occurs 2
'http://targetedattacks.trendmicro.com/img/flags/Japanese.png' occurs 2
'http://targetedattacks.trendmicro.com/img/flags/China.png' occurs 2
'http://targetedattacks.trendmicro.com/img/flags/Hungarian.png' occurs 2
'http://targetedattacks.trendmicro.com/img/flags/Polish.png' occurs 2
'http://targetedattacks.trendmicro.com/img/flags/Braziilian.png' occurs 2
'http://www.faqs.org/favicon.ico' occurs 2
'http://192.168.254.113/mutillidae/index.php?page=%%32%65%%32%65%2f%%32%65%%32%65%2f%2fetc%2fpasswd' occurs 2
'http://pubimage.360doc.com/index7/nlogo.jpg' occurs 2
'http://www.people.com.cn/img/2020peopleinde

'http://vas.vtiger.in/libraries/client/jsaddons/autolink-min.js' occurs 2
'http://vas.vtiger.in/libraries/client/jsaddons/purify.js' occurs 2
'http://www.washington.edu/static/home/wp-content/themes/uw-2014/assets/images/boundless.png' occurs 2
'http://vas.vtiger.in/libraries/client/jsaddons/twitter-text.min.js' occurs 2
'http://vas.vtiger.in/libraries/client/underscorejs/underscore-min.js' occurs 2
'http://vas.vtiger.in/libraries/client/jquery/select2/select2.min.js' occurs 2
'http://vas.vtiger.in/libraries/client/angularjs-ui/angular-ui.js' occurs 2
'http://vas.vtiger.in/libraries/client/jquery/jquery.timeago.js' occurs 2
'http://vas.vtiger.in/libraries/client/jquery/jquery.truncator.min.js' occurs 2
'http://vas.vtiger.in/libraries/client/jquery/jlinq/jlinq.js' occurs 2
'http://vas.vtiger.in/libraries/client/jquery/jlinq/jlinq.jquery.js' occurs 2
'http://vas.vtiger.in/libraries/client/bootstrap/plugins/notify/js/bootstrap-notify.js' occurs 2
'http://www.washington.edu/static/home/wp-

'http://www.people.com.cn/img/2020peopleindex/img/copy_icon2.png' occurs 2
'http://www.people.com.cn/NMediaFile/2022/1227/MAIN202212271458196177173655165.png' occurs 2
'http://www.people.com.cn/img/2020peopleindex/img/logormw1.png' occurs 2
'http://www.people.com.cn/img/2020peopleindex/img/8.jpg' occurs 2
'http://www.people.com.cn/img/2020peopleindex/img/10.jpg' occurs 2
'http://www.people.com.cn/img/2020peopleindex/img/logormy.png' occurs 2
'http://www.people.com.cn/img/2020peopleindex/img/logohw.png' occurs 2
'http://www.people.com.cn/img/2020peopleindex/img/logoyjy.png' occurs 2
'http://www.people.com.cn/img/2020peopleindex/img/copy_py2.png' occurs 2
'http://www.people.com.cn/img/2020peopleindex/img/copy_icon1.png' occurs 2
'http://www.faqs.org/style/faqs.css' occurs 2
'http://192.168.254.113/mutillidae/index.php?page=%%32%65%%32%65%2f%%32%65%%32%65%2f%%32%65%%32%65%2f%%32%65%%32%65%2f%%32%65%%32%65%2f%%32%65%%32%65%2f%2fetc%2fpasswd' occurs 2
'http://www.socialstudieshelp.com/' occ

'http://static02.babytreeimg.com/concat/??/img/js/jquery-1.6.4.1.min.js,/img/bui/util/core.js,/static/bui/extension/header/globalHeader.js,/img/js/header2011.js,/img/js/dateSelectSolo.js?ver=1542283745' occurs 2
'http://swiat-kamienia.pl/plugins/system/jcemediabox/css/jcemediabox.css?version=114' occurs 2
'http://e1.o.lencr.org/MFMwUTBPME0wSzAJBgUrDgMCGgUABBTvkAFw3ViPKmUeIVEf3NC7b1ErqwQUWvPtK%2Fw2wjd5uVIw6lRvz1XLLqwCEgQUiU4IoHTqc2ziCte44TBm%2BA%3D%3D' occurs 2
'http://swiat-kamienia.pl/modules/mod_fpss/tmpl/Movies/css/template.css.php?width=425&height=300&sidebarWidth=240&timer=1&thumbnailViewportWidth=65&thumbnailViewportHeight=40&mid=158' occurs 2
'http://swiat-kamienia.pl/modules/mod_sptab/assets/css/custom.css.php?id=137' occurs 2
'http://ocsp.globalsign.com/ca/gsatlasr3dvtlsca2023q2/MFEwTzBNMEswSTAJBgUrDgMCGgUABBQliXE6aHWZ%2FruuxapKUSXjiSRU7wQUxqJhE0UVzJCPusGsvMw7DQ7Q3gYCEAF%2BAPGuTNK8KK4KMrFVbNE%3D' occurs 2
'http://ocsp2.globalsign.com/rootr3/MFEwTzBNMEswSTAJBgUrDgMCGgUABBT1nGh%

In [58]:
df

,No.,Arrival Time,Time,Source,Destination,Protocol,Frame Length,Info,Full request URI,Label
0,1,"Sep 18, 2023 23:27:26.023406000 Malay Peninsul...",0.000000,192.168.95.128,192.168.95.129,HTTP,359,GET /dvwa/vulnerabilities/fi/..%2f..%2f..%2f.....,http://192.168.95.129/dvwa/vulnerabilities/fi/...,1
1,2,"Sep 18, 2023 23:27:26.326025000 Malay Peninsul...",0.302619,192.168.95.128,192.168.95.129,HTTP,302,GET /dvwa/vulnerabilities/fi/..%5cdeep_travers...,http://192.168.95.129/dvwa/vulnerabilities/fi/...,1
2,3,"Sep 18, 2023 23:27:26.628313000 Malay Peninsul...",0.604907,192.168.95.128,192.168.95.129,HTTP,253,GET /dvwa/vulnerabilities/fi/..%5c..%5cdeep_tr...,http://192.168.95.129/dvwa/vulnerabilities/fi/...,1
3,4,"Sep 18, 2023 23:27:26.931207000 Malay Peninsul...",0.907801,192.168.95.128,192.168.95.129,HTTP,333,GET /dvwa/vulnerabilities/fi/..%5c..%5c..%5cde...,http://192.168.95.129/dvwa/vulnerabilities/fi/...,1
4,5,"Sep 18, 2023 23:27:27.233910000 Malay Peninsul...",1.210504,192.168.95.128,192.168.95.129,HTTP,309,GET /dvwa/vulnerabilities/fi/..%5c..%5c..%5c.....,http://192.168.95.129/dvwa/vulnerabilities/fi/...,1
...,...,...,...,...,...,...,...,...,...,...
12999,43029,"Oct 11, 2023 15:05:30.662771357 Malay Peninsul...",2581.682975,10.0.2.15,91.199.212.90,HTTP,389,GET / HTTP/1.1,http://sectigo.com/,0
13000,46451,"Oct 11, 2023 15:06:55.022809323 Malay Peninsul...",2666.043013,10.0.2.15,104.18.22.208,HTTP,388,GET / HTTP/1.1,http://genius.com/,0
13001,46653,"Oct 11, 2023 15:07:06.026505998 Malay Peninsul...",2677.046710,10.0.2.15,18.172.21.121,HTTP,391,GET / HTTP/1.1,http://tremorhub.com/,0
13002,46754,"Oct 11, 2023 15:07:08.581745984 Malay Peninsul...",2679.601950,10.0.2.15,107.181.87.5,HTTP,342,GET /icons/ubuntu-logo.png HTTP/1.1,http://icio.us/icons/ubuntu-logo.png,0


-------------------------------------------------

# Remove any unecessary strings in the Full request URI
Since the sessions were conducted in a controlled environment, it will be redundant as all traffic were defined. There were also strings that were unecessary that will affect the quality of dataset.

- Replace any unecessary strings
- Shuffle the rows

In [59]:
# Define the replacement patterns. Remove all unecessary wordings in the payload URI
# IP Mach1 (elias), IP Mach2 (elias), IP Mach3 (bryan)
patterns = [r'deep_traversal.txt', r'http://192.168.95.129/', r'http://172.24.30.219/', r'http://192.168.254.113', 
           r'http://192.168.254.117']


# Apply replacement patterns in a loop
for pattern in patterns:
    df['Full request URI'] = df['Full request URI'].str.replace(pattern, '', regex=True)
    
# Shuffle the rows
df = df.sample(frac=1.0, random_state=42)

In [60]:
# Show all the column names of the dataframe
columns = df.columns
columns

Index(['No.', 'Arrival Time', 'Time', 'Source', 'Destination', 'Protocol',
       'Frame Length', 'Info', 'Full request URI', 'Label'],
      dtype='object')

--------------------------------

# Feature Selection
Dropping columns that are not relevant to LFI attacks or Layer 7 attacks and to the nature of the attacks as it was conducted only in a controlled environment.

In [61]:
df = df.drop(labels=['No.', 'Arrival Time', 'Time', 'Source', 'Destination', 'Protocol', 'Frame Length', 'Info'], axis=1)

In [62]:
df

,Full request URI,Label
12577,http://thenextweb.com/in/2015/03/24/india-u-tu...,0
11604,http://slickdeals.net/forums/forumdisplay.php?...,0
4526,dvwa/vulnerabilities/fi/%c0%2e%c0%2e%c1%9c%c0%...,1
7613,http://www.piutetrailrvpark.com/wp-content/the...,0
4353,dvwa/vulnerabilities/fi/%252e%252e%c1%pc%252e%...,1
...,...,...
11964,http://cox.com/residential/support/alert.cox?a...,0
5191,/mutillidae/index.php?page=%2e%2e%5c%2e%2e%5c%...,1
5390,/mutillidae/index.php?page=%2e%2e%2e%5c%2e%2e%...,1
860,dvwa/vulnerabilities/fi/.?%f0%80%80%af.?%f0%80...,1


----------------------

# Feature Engineering and Extraction
Using the Full request URI to help distinguish informative feature that can be used in detecting LFI attacks. Using the attack vectors that was defined in the Scope of the Study, we used methods to determine if there were attacks that behave on the following:

1. Directory Traversal
2. File Path Manipulation
3. Null Byte Poisoning
4. PHP/Data Wrapper
5. Double Encoding

### 1. Directory Traversal
Check if the Full request URI has a path traversal pattern.

##### Pseudocode
START

IMPORRT re library

DECLARE LFI traversal patterns in 'traversal_pattern' variable


FUNCTION has_traversal(uri)
	IF uri has any traversal pattern/s 
		THEN return 1
	ELSE return 0


PUT 'has_traversal' return value to dataframe under 'HasDotDotPwn' column

In [63]:
import re

# Regular expression pattern to check for traversal sequences. This ensures that the special characters to detect is bounded between slashes
traversal_pattern = r'(?:^|[/?])((\.\./|\?\?/|\?\./|\.?/)+)(?:$|[/?])'

# Function to check for traversal sequences in a URI and return 1 if found, 0 otherwise
def has_traversal(uri):
    match = re.search(traversal_pattern, uri)
    return 1 if match and len(match.group(1)) > 0 else 0

df['HasDirectoryTraversal'] = df['Full request URI'].apply(has_traversal)


### 2. File Path Manipulation
Determines if the URI contains the pattern which tries to exploit and access such directory and commands

START

IMPORRT re library

DECLARE LFI path manipulation in 'exploit_pattern' variable


FUNCTION has_exploit(uri)
	IF uri has any exploit pattern/s 
		THEN return 1
	ELSE return 0


PUT 'has_exploit' return value to dataframe under 'HasFilePathManipulation' column



In [64]:
import re

# Regular expression pattern to detect platform-specific exploits
exploit_pattern = r'(etc|cmd.exe|powershell.exe|c:\\\\|c%3a%5c|/etc/passwd|/etc/shadow|proc/self/environ)'

# Function to check for platform-specific exploits in a URI and return 1 if found, 0 otherwise
def has_exploit(uri):
    return 1 if re.search(exploit_pattern, uri, re.IGNORECASE) else 0

# Apply the has_platform_exploit function to each row in the "Full request URI" column
df['HasFilePathManipulation'] = df['Full request URI'].apply(has_exploit)

### 3. Null Byte Poisoning
This method will check if there were null byte attacks (an LFI attack vector) present in the URI.

START

IMPORRT re library

DECLARE LFI null byte poisoning patterns in 'exploit_pattern' variable


FUNCTION has_patterns(uri)
	IF uri has any null byte pattern/s 
		THEN return 1
	ELSE return 0


PUT 'has_patterns' return value to dataframe under 'HasNullBytePoisoning' column

In [65]:
# Define a list of patterns to recognize (e.g., null byte)
patterns_to_recognize = [r'%00', r'\x00', r'0x']

# Function to check for the presence of patterns in a URI
def has_patterns(uri, patterns):
    for pattern in patterns:
        if re.search(pattern, uri):
            return 1
    return 0

# Apply the has_patterns function to each row in the "Full request URI" column
df['HasNullBytePoisoning'] = df['Full request URI'].apply(has_patterns, patterns=patterns_to_recognize)

### 4. PHP/Data Wrapper Abuse
Check if these patterns are existing in the URI that may abuse the unsanitze web application throught directory or file path traversal that may result into LFI exploitation and deliberation.


START

IMPORRT re library

DECLARE LFI data wrapper patterns in 'keywords_to_detect' variable


FUNCTION has_keyword(uri)
	IF uri has any data wrapper patterns
		THEN return 1
	ELSE return 0


PUT 'has_keyword' return value to dataframe under 'HasDataWrapperAbuse' column


In [66]:
# Keywords or patterns to detect in the URI
datawrapper_patterns = ['bin/bash', 'var/log', 'etc/hosts','etc/passwd', 'etc/shadow', 'file:///', 'php://', 'zip://' ,'passwd']

# Function to check for the presence of keywords in a URI and return 1 if found, 0 otherwise
def has_datawrapper(uri):
    for keyword in datawrapper_patterns:
        if re.search(re.escape(keyword), uri):
            return 1
    return 0

# Apply the has_keywords function to each row in the "Full request URI" column
df['HasWrapperAbuse'] = df['Full request URI'].apply(has_datawrapper)

### 5. Double Encoded Sequences
Check if encoding patterns occurs on the Full request URI column since this technique is commonly used for LFI attacks.

START

IMPORRT re library

DECLARE LFI encoded sequence patterns in 'encoded_sequence_pattern' variable


FUNCTION has_encoded_sequence(uri)
	IF uri has any encoded sequence/s 
		THEN return 1
	ELSE return 0


PUT 'has_encoded_sequence' return value to dataframe under 'HasDoubleEncoding' column


In [67]:
# Regular expression pattern to detect encoded sequences (e.g., %2e, %2f, %5c, etc.)
encoded_sequence_pattern = r'%[0-9a-fA-F]{1}[0-9a-fA-F]*|%u[0-9a-fA-F]{4}'

# Function to check for encoded sequences in a URI and return 1 if found, 0 otherwise
def has_encoded_sequences(uri):
    return 1 if re.search(encoded_sequence_pattern, uri) else 0

# Apply the has_encoded_sequences function to each row in the "Full request URI" column
df['HasDoubleEncoding'] = df['Full request URI'].apply(has_encoded_sequences)

### 6. URI Character Count
Counts the letter, value, and special character from Full request URI

START

FUNCTION count_characters(cell_value):
    remove spaces in the URI and
    return URI length

PUT 'count_character' return value to its specified column in the dataframe


In [68]:
# Function to count characters in a cell and return the count
def count_characters(cell_value):
    return len(str(cell_value).replace(" ",""))

# Apply the function to the "Full request URI" column and create a new "URILength" column
df['URILength'] = df['Full request URI'].apply(lambda x: count_characters(x))

### 7. URI Special Character/s
Counts the special character occurence in the Full request URI as LFI attacks are prominent in using special characters to exploit the vulnerability. 

START

DECLARE patterns under 'special_char_pattern' to detect by the method

DEFINE count_special_characters(uri)
    return occurence value of special characters from URI
    
PUT 'count_special_characters' return value to its corresponding column in dataframe

In [69]:
# Regular expression pattern to detect special characters
special_char_pattern = r'[!@#$%^&*()_+{}\[\]:;<>,.?/~\\-]'

# Function to count special characters in a URI
def count_special_characters(uri):
    return len(re.findall(special_char_pattern, uri))

# Apply the count_special_characters function to each row in the "URI" column
df['SpecialCharCount'] = df['Full request URI'].apply(count_special_characters)

### 8. URI Path Depth
Count how many directories was traversed in the URI. THis can be referenced by counting the '/' (slash) connotation.


START

FUNCTION calculate_path_depth(uri)
    do splice the URI by '/' to get path components
    do calculate how many slashes or directories traversed in the URI
    return value
    
PUT calculate_path_depth return value to its corresponding column in the dataframe

In [70]:
# Function to calculate the path depth of a URI
def calculate_path_depth(uri):
    # Split the URI by '/' to extract path components
    path_components = uri.split('/')
    
    # Calculate the number of directories traversed (excluding empty strings)
    path_depth = len([component for component in path_components if component and component != '..'])
    
    return path_depth

# Apply the calculate_path_depth function to each row in the "URI" column
df['PathDepth'] = df['Full request URI'].apply(calculate_path_depth)

### 9. Tokenizer 
Process to breakdown URI into chunks of strings to make meaningful components or tokens. Tokenizing URIs involves separating these components and often ignoring special characters and delimiters. It may separate it on the following:

- Scheme (protocol)
- Host 
- Path
- Query Parameter
- Fragment Identifier

Special characters are often not inclusive in this tokenization process. 


START

IMPORT nltk library

FUNCTION token_count(uri,tokenizer)
    return token length value
    
INITIALIZE tokenizer

do apply the tokenizer and the function
place the return value from the function to its corresponding column in the dataframe


In [71]:
import nltk
# Function to tokenize a URI and count the number of tokens
def token_count(uri, tokenizer):
    tokens = tokenizer(uri)
    return len(tokens)

# Apply the token_count function to each row in the "URI" column using nltk's word tokenizer
tokenizer = nltk.word_tokenize  # You can use this for word-level tokenization
df['TokenCount'] = df['Full request URI'].apply(token_count, tokenizer=tokenizer)

### 10. Attack Vector 
Checks if the specified columns has a value and return if it has.

START

do check if the following columns has a boolean value
    if it has a value, return 1
    ELSE 0
    
do put the return value to its corresponding column in the dataframe


In [72]:
# Check if any of the specified columns have a value (non-zero) in each row
df['HasAttackVector'] = df[['HasDirectoryTraversal', 'HasWrapperAbuse', 'HasDoubleEncoding', 
                            'HasFilePathManipulation', 'HasNullBytePoisoning']].any(axis=1).astype(int)

In [73]:
df

,Full request URI,Label,HasDirectoryTraversal,HasFilePathManipulation,HasNullBytePoisoning,HasWrapperAbuse,HasDoubleEncoding,URILength,SpecialCharCount,PathDepth,TokenCount,HasAttackVector
12577,http://thenextweb.com/in/2015/03/24/india-u-tu...,0,0,0,0,0,0,89,19,8,3,0
11604,http://slickdeals.net/forums/forumdisplay.php?...,0,0,0,0,0,0,88,11,4,11,0
4526,dvwa/vulnerabilities/fi/%c0%2e%c0%2e%c1%9c%c0%...,1,0,1,0,1,1,93,23,4,41,1
7613,http://www.piutetrailrvpark.com/wp-content/the...,0,0,0,0,0,0,92,15,7,5,0
4353,dvwa/vulnerabilities/fi/%252e%252e%c1%pc%252e%...,1,0,1,0,1,1,103,21,4,37,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11964,http://cox.com/residential/support/alert.cox?a...,0,0,0,0,0,0,89,13,5,5,0
5191,/mutillidae/index.php?page=%2e%2e%5c%2e%2e%5c%...,1,0,1,1,1,1,129,32,2,59,1
5390,/mutillidae/index.php?page=%2e%2e%2e%5c%2e%2e%...,1,0,1,0,1,1,126,34,2,63,1
860,dvwa/vulnerabilities/fi/.?%f0%80%80%af.?%f0%80...,1,0,0,0,0,1,80,27,4,38,1


# Save to a CSV File

In [74]:
# Use only if you want to put in a combined csv file
df.to_csv("preprocessdata++.csv", index=False)